La primera celda incluye las declaraciones de funciones auxiliares así como la función de generación de correspondientes a variables significativas para añadir al RDD base.

In [ ]:
#Imports
import pyspark.mllib.linalg.distributed as dist
import random


# Tuple for generation of breast cancer related data
# its based on 3 values, gen1,gen2 and age
#[1:-1] sets prob of cancer depending on gen1 and gen2 [gen1=1,gen2=1,gen1&gen2=1,gen1&gen2=0]
#[-1] sets an age control, if age<[-1] cancer goes to 0
probabilityTuple=(0.7,0.4,0.90,0.02,0.4)


def strToInt(x):
    newList=[]
    for value in x:
        newList.append(int(float(value.replace('[', ''))))   
    return newList

# Recives path to textfile(csv,zip,ect) containing 2 dimensional data  
# Returns Rdd of (cord1,cord2,value), with cords going from 0 to max value 
#
# Note: both "replace" values are for python2, not needed in python3
# Note2: for reference, not used in this secuence
def Csv_File_to_Coordinate_Rdd(filename):
    return (sc
    .textFile(filename)
    .zipWithIndex()
    .flatMap(lambda x: ([(x[1],j[0],float(j[1]))for j in enumerate(x[0].replace('[','0').replace(']','0').split(','))]))
    .filter(lambda x: x[-1] != 0)  
)
  
def List_to_Coordinate_Rdd(listName):
    return (sc.parallelize(listName)
          .zipWithIndex()
          .flatMap(lambda x: ([(x[1],j[0],float(j[1]))for j in enumerate(x[0])]))
          .filter(lambda x: x[-1] != 0))

def Rdd_to_Coordinate_Rdd(rddName):
    return (rddName.zipWithIndex()
          .flatMap(lambda x: ([(x[1],j[0],float(j[1]))for j in enumerate(x[0])]))
          .filter(lambda x: x[-1] != 0))  

  
#normal base balues tuple should have the size of the data you want to add
#for example for 3 values (1,1,1)
#recieves also a tuple of functions to calculate them
#and how many values we want to add that must be lower than the CoordinateValueCount
def getExtraCoordinateValues(probabilityTuple,numberAlteredPoints):
    TempList=[]
    for n in range(0,numberAlteredPoints):
        T1=list(map(lambda x:x*random.random(),(1,1,1,1)))
        T1[0]=int(T1[0]<0.05)#prob de esta alteracion
        T1[1]=int(T1[1]<0.05)#prob de esta alteracion
    
        if T1[0]==1 and T1[1]==1:
            T1[3]=int(T1[3]<probabilityTuple[2])
        elif T1[0]==1:
            T1[3]=int(T1[3]<probabilityTuple[0])
        elif T1[1]==1: #T2==1 true
            T1[3]=int(T1[3]<probabilityTuple[1])
        else:
            T1[3]=int(T1[3]<probabilityTuple[3])
        if T1[2]<probabilityTuple[4]: 
            T1[3]=0  #low age limit 
        #modificacion para usar enteros
            T1[2]=0
        else:
            T1[2]=1
        TempList.append(T1)
    return TempList  

La siguiente celda incluye el código de generacion de RDDs aleatorios

In [ ]:
import random
import numpy as np

#size of data to generate
numberOfElementsPerRow=1000
numberOfRows=100000

#%0s %1s
distribution_0_1=[0.95,0.05]

#Random value rdd generation
dataMatrix=[]
for i in range(0,numberOfRows):
    dataMatrix.append([0])
RandomRdd=sc.parallelize(dataMatrix).map(lambda x: np.random.choice([0,1], size=numberOfElementsPerRow, replace=True, p=distribution_0_1).tolist())
#output in RandomRdd


La siguiente casilla utiliza el Rdd aleatorio en RandomRdd, genera los nuevos valores para cada fila y escribe la salida en la ruta de destino.

In [ ]:
import pyspark
from pyspark.mllib.regression import LabeledPoint

outputPath="D:/SparkFiles/100Kx1K0.05PIntAge"

size=len(RandomRdd.take(1)[0])+4
Random_Coordinate_Rdd=(RandomRdd
                       .map(lambda x: x+getExtraCoordinateValues(probabilityTuple,1)[0])
                       .map(lambda x: LabeledPoint(x[-1],pyspark.mllib.linalg.SparseVector(size,range(size),x[:]))))
                            
pyspark.mllib.util.MLUtils.saveAsLibSVMFile(Random_Coordinate_Rdd,outputPath)